In [1]:
from hepmc import *
import numpy as np

In [2]:
sarge = densities.Sarge(2, 4, 100.)

In [3]:
eeqq2g = densities.ee_qq_ng(2, 100., 5., .3)

In [4]:
rambo_mapping = phase_space.RamboOnDiet(100., 4)
mapped = phase_space.MappedDensity(eeqq2g, rambo_mapping)

In [ ]:
%time sarge_sample = ImportanceMC(sarge)(eeqq2g.pdf, 10000)

Event 1	(avg. trials per event: 2.000000)
Event 2	(avg. trials per event: 12.000000)	XS = 81.059722 pb +- ( 58.740551 pb = 72.465769 %)
Event 3	(avg. trials per event: 25.333333)	XS = 57.776309 pb +- ( 25.172125 pb = 43.568248 %)
Event 4	(avg. trials per event: 25.500000)	XS = 43.193637 pb +- ( 22.682008 pb = 52.512384 %)
Event 5	(avg. trials per event: 20.800000)	XS = 44.286931 pb +- ( 23.217667 pb = 52.425550 %)
Event 6	(avg. trials per event: 22.000000)	XS = 35.811904 pb +- ( 18.919679 pb = 52.830699 %)
Event 7	(avg. trials per event: 21.571429)	XS = 31.358251 pb +- ( 17.106163 pb = 54.550758 %)
Event 8	(avg. trials per event: 25.125000)	XS = 29.439374 pb +- ( 12.965610 pb = 44.041732 %)
Event 9	(avg. trials per event: 22.666667)	XS = 36.995861 pb +- ( 13.404858 pb = 36.233399 %)
Event 10	(avg. trials per event: 25.900000)	XS = 29.288183 pb +- ( 10.938170 pb = 37.346700 %)
Event 20	(avg. trials per event: 18.600000)	XS = 46.396136 pb +- ( 13.856883 pb = 29.866459 %)
Event 30	(avg. t

In [ ]:
densities.export_hepmc(100., sarge_sample, "../samples/qcd/2-4/sarge_weighted.hepmc")

In [ ]:
bound = sarge_sample.weights.max()

In [ ]:
%%time
# acceptance-rejection sampler
sarge_sample_unweighted = np.empty((1000, 16))
for i in range(1000):
    aprob = 0
    while np.random.random() > aprob:
        proposal = sarge.proposal()
        f = eeqq2g.pdf(proposal)
        g = sarge.proposal_pdf(None, proposal)
        aprob = f/(bound*g)
        if aprob > 1:
            print("bound too low")
    sarge_sample_unweighted[i] = proposal
sarge_sample_unweighted = Sample(data=sarge_sample_unweighted, weights=np.full(1000, 1./1000))

In [ ]:
densities.export_hepmc(100., sarge_sample_unweighted, "../samples/qcd/2-4/sarge_unweighted.hepmc")

In [ ]:
rambo_sample = PlainMC(8)(mapped, 10000)
rambo_sample.integral, rambo_sample.integral_err

In [ ]:
ys = mapped.pdf(rambo_sample.data)
max_y = ys.max()

In [ ]:
rambo = AcceptRejectSampler(mapped, max_y, 8)
%time rambo_sample = rambo.sample(1000)

In [ ]:
rambo_sample.data = rambo_mapping.map(rambo_sample.data)
rambo_sample.weights = np.full(rambo_sample.size, 1./rambo_sample.size)

In [ ]:
densities.export_hepmc(100., rambo_sample, "../samples/qcd/2-4/rambo.hepmc")

In [ ]:
# just a local sampler (rambo)
met = DefaultMetropolis(8, mapped, proposals.Gaussian(8, .01))
%time met_sample = met.sample(10000, np.random.rand(8))

In [ ]:
met_sample.data = rambo_mapping.map(met_sample.data)
met_sample.weights = np.full(met_sample.size, 1./met_sample.size)
densities.export_hepmc(100., met_sample, "../samples/qcd/2-4/metropolis.hepmc")

In [ ]:
# now sarge + local in MC3
importance = DefaultMetropolis(4 * 4, eeqq2g, proposal=sarge)  # using sarge, full momentum sample space
local = DefaultMetropolis(8, mapped, proposals.Gaussian(8, .01))  # using rambo, rambo sample space
update = MixingMarkovUpdate(8, [importance, local], target=mapped,
                            out_maps={0: rambo_mapping.map_inverse},
                            in_maps={0: rambo_mapping.map}) # rambo sample space, important to set target explicitly
%time mc3_sample = update.sample(10000, np.random.rand(8))

In [ ]:
mc3_sample.data = rambo_mapping.map(mc3_sample.data)
mc3_sample.weights = np.full(mc3_sample.size, 1./mc3_sample.size)
densities.export_hepmc(100., mc3_sample, "../samples/qcd/2-4/mc3.hepmc")